In [13]:
# creating paths to src and data folders in the repo
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"
dump_path = data_path / 'initial_dump'

# train test split paths
train_path = data_path / 'fsm_tts/train'
test_path = data_path / 'fsm_tts/test'

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# basic imports for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# silence max image size warning
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 

# import modeling packages
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# modeling metrics
from sklearn.metrics import classification_report, confusion_matrix

changes to data generators:
    
    test batch size 49 -> 33

In [4]:
train_generator = ImageDataGenerator().flow_from_directory(str(train_path),
                                                           target_size=(150, 150),
                                                           batch_size=100,
                                                           class_mode='categorical')
test_generator = ImageDataGenerator().flow_from_directory(str(test_path),
                                                          target_size=(150, 150),
                                                          batch_size=33,
                                                          class_mode='categorical')

Found 5700 images belonging to 6 classes.
Found 363 images belonging to 6 classes.


changes to model architecture:
    
    1st conv2d layer num_filters 20 -> 30
    
    1st dense layer nodes 20 -> 40
    
    num conv2d layers 1 -> 2

I think i may have may it too simple and it's resulted in extremely poor accuracy

In [14]:
model = models.Sequential()
model.add(layers.Conv2D(30, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(30, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 148, 30)      840       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 30)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 72, 30)        8130      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 36, 30)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 38880)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 40)                1555240   
_________________________________________________________________
dense_5 (Dense)              (None, 6)                

changes to fit:
    
    steps_per_epoch 61 -> 57
    
    validation_steps 19 -> 11
    
    epochs 3

In [16]:
model.fit(
        train_generator,
        steps_per_epoch=57,
        epochs=3,
        validation_data=test_generator,
        validation_steps=11)

Epoch 1/3
31/57 [===============>..............] - ETA: 4:53 - loss: 78.1807 - accuracy: 0.1845

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


57/57 [==============================] - 679s 12s/step - loss: 43.3603 - accuracy: 0.2026 - val_loss: 1.8480 - val_accuracy: 0.2369
Epoch 2/3
57/57 [==============================] - 674s 12s/step - loss: 1.7380 - accuracy: 0.2589 - val_loss: 1.8113 - val_accuracy: 0.2397
Epoch 3/3
57/57 [==============================] - 662s 12s/step - loss: 1.6242 - accuracy: 0.3232 - val_loss: 1.8145 - val_accuracy: 0.2727


In [17]:
confusion_matrix(test_generator.classes, np.argmax(model.predict(test_generator), axis=1))

array([[11,  7, 34,  2,  2,  3],
       [12,  6, 31,  1,  4,  5],
       [12, 15, 32,  5,  2,  2],
       [10,  9, 27,  5,  2,  6],
       [10, 10, 33,  0,  4,  2],
       [10,  4, 27,  5,  7,  6]])

okay finally looks like the right direction. next going to add back in some complexity and droppout to reduce overfitting as well as potentially increasing number of epochs